<a href="https://colab.research.google.com/github/dcownden/PerennialProblemsOfLifeWithABrain/blob/main/sequences/P1C1_BehaviourAsPolicy/student/P1C1_Sequence1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> &nbsp; <a href="https://kaggle.com/kernels/welcome?src=https://raw.githubusercontent.com/dcownden/PerennialProblemsOfLifeWithABrain/main/sequences/P1C1_BehaviourAsPolicy/student/P1C1_Sequence1.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open in Kaggle"/></a>

# **Part 1 Behaviour, Environments and Optimization: Evolution and Learning**

### **Animals are adapted to their specific environments; their behaviour is best understood within the context of their evolutionary environment.**

### Objective: Part 1 of the book aims to introduce the fundamental concepts of
* ### **environment**, where an organism lives;
* ### **behaviour**, what the organism does there;
* ### **optimization**, how learning and evolution shape an organism's behaviour to make it better suited to its environment.

___
# Chapter 1.1 Behaviour as a Policy in an Environmental Context

### Objective: This chapter develops examples of how behaviour is described and evaluated in relation to its [goodness](## "This is a very loaded term, to be unpacked carefully later") within a specific environmental niche.

You will learn:
*   What is a policy? A policy is a formalization of behaviour as a function that takes an organism's experiences of their environment as an input and outputs the organism's actions.
*   What is a good policy? The rewards and other environmental signals, resulting from the organism's actions in the environment, are integrated into a Loss/Objective function to evaluatate, and potentially improve, a policy.
*   What is stochasticity? Both the environment and an organism's behavior can contain random elements. This randomness can pose challenges when evaluating policies, as it becomes difficult to determine whether poor outcomes are due the policy itself or simply bad luck.



___
# **Sequence 1.1.1: Gridworld Introduction**

### Objective: In this sequence, we will create a simple environment-organism system to demonstrate how an organism's **behaviour**, within an **environment**, can be evaluated using **rewards**. We will also explore how intelligent behaviour can lead to better outcomes and how **randomness** can make evaluation of behaviour difficult.

# Setup

You don't need to worry about how the code in the following cells works, but you do need to run these cells, in order.

In [ ]:
# @title Dependency Downloads and Installs
# @markdown You don't need to worry about how this code works – but you do need to **run the cell**
!pip install ipympl vibecheck datatops > /dev/null 2> /dev/null #google.colab

In [ ]:
# @title Imports and Random Number Generation Setup
# @markdown You don't need to worry about how this code works – but you do need to **run the cell**
import ipywidgets as widgets
import functools
import threading
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt
import time
import asyncio
import logging
#import jax

from vibecheck import DatatopsContentReviewContainer

IN_COLAB = False
try:
  print('Running in colab')
  from google.colab import output
  output.enable_custom_widget_manager()
  #from google.colab import output as colab_output
  #colab_output.enable_custom_widget_manager()
  IN_COLAB = True
except:
  print('Not running in colab')


# set numpy random seed
SEED = 2021
rng = np.random.default_rng(seed=SEED)

In [ ]:
# @title Plotting Setup and Functions
# @markdown You don't need to worry about how this code works – but you do need to **run the cell**

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib widget
plt.style.use("https://raw.githubusercontent.com/dcownden/PerennialProblemsOfLifeWithABrain/main/pplb.mplstyle")
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

def make_grid(num_rows, num_cols):
  """Plots an n_rows by n_cols grid with cells centered on integer indices and
  returns fig and ax handles for futher use
  Args:
    num_rows (int): number of rows in the grid (vertical dimension)
    num_cols (int): number of cols in the grid (horizontal dimension)

  Returns:
    fig (matplotlib.figure.Figure): figure handle for the grid
    ax: (matplotlib.axes._axes.Axes): axes handle for the grid
  """
  fig, ax = plt.subplots(figsize = (7,6), layout='constrained')
  ax.spines[['right', 'top']].set_visible(True)
  ax.set_xticks(np.arange(0, num_cols, 1))
  ax.set_yticks(np.arange(0, num_rows, 1))
  # Labels for major ticks
  ax.set_xticklabels(np.arange(0, num_cols, 1),fontsize=8)
  ax.set_yticklabels(np.arange(0, num_rows, 1),fontsize=8)

  # Minor ticks
  ax.set_xticks(np.arange(0.5, num_cols-0.5, 1), minor=True)
  ax.set_yticks(np.arange(0.5, num_rows-0.5, 1), minor=True)

  ax.xaxis.tick_top()

  # Gridlines based on minor ticks
  ax.grid(which='minor', color='grey', linestyle='-', linewidth=2, alpha=0.5)

  # Remove minor ticks
  ax.tick_params(which='minor', bottom=False, left=False)

  ax.set_xlim(( -0.5, num_cols-0.5))
  ax.set_ylim(( -0.5, num_rows-0.5))
  ax.invert_yaxis()
  fig.canvas.header_visible = False
  fig.canvas.toolbar_visible = False
  fig.canvas.resizable = False
  fig.canvas.footer_visible = False
  fig.canvas.draw()
  return fig, ax

def plot_food(fig, ax, rc_food_loc, food=None):
  """
  Plots "food" on a grid implied by the given fig, ax arguments

  Args:
    fig, ax: matplotlib figure and axes objects
    rc_food_loc: ndarry(int) of shape (N:num_food x 2:row,col)
    food: a handle for the existing food matplotlib PatchCollenction object
    if one exists
  Returns:
    a handle for matplotlib PathCollection object of food scatter plot, either
    new if no handle was passed or updated if it was
  """
  # if no PathCollection handle passed in:
  if food is None:
    food = ax.scatter([], [], s=150, marker='o', color='red', label='Food')
  rc_food_loc = np.array(rc_food_loc, dtype=int)
  #matrix indexing convention is is [row-vertical, col-horizontal]
  #plotting indexing convention is (x-horizontal,y-vertical), hence flip
  food.set_offsets(np.fliplr(rc_food_loc))
  return food

def plot_critter(fig, ax, rc_critter_loc, critter=None):
  """
  Plots "critters" on a grid implied by the given fig, ax arguments

  Args:
    fig, ax: matplotlib figure and axes objects
    rc_critter_loc: ndarry(int) of shape (N:num_critters x 2:row,col)
    critter: a handle for the existing food matplotlib PatchCollenction object
    if one exists
  Returns:
    a handle for matplotlib PathCollection object of critter scatter plot,
    either new if no handle was passed in or updated if it was.
  """
  if critter is None:
    critter = ax.scatter([], [], s=250, marker='h', color='blue', label='Critter')
  #matrix indexing convention is is [row-vertical, col-horizontal]
  #plotting indexing convention is (x-horizontal,y-vertical), hence flip
  critter.set_offsets(np.flip(rc_critter_loc))
  return critter

In [ ]:
# @title Simulation Functions, Variables and widgets
# @markdown You don't need to worry about how this code works – but you do need to **run the cell**

def init_loc(n_rows, n_cols, num):
  """
  Samples random 2d grid locations, without replacement,

  Returns:
    int_loc: ndarray(int) of flat indices for a grid
    rc_index: (ndarray(int), ndarray(int)) a pair of arrays the first
      giving the row indices, the second giving the col indices, useful
      for indexing an n_rows by n_cols numpy array
    rc_plotting: ndarray(int) num x 2, same rc coordinates but structured
      in the way that matplotlib likes
  """
  int_loc = np.random.choice(n_rows * n_cols, num, replace=False)
  rc_index = np.unravel_index(int_loc, (n_rows, n_cols))
  rc_plotting = np.fliplr(np.array(rc_index).T)
  return int_loc, rc_index, rc_plotting


class GridworldBoard():
  """
  A collection methods and parameters of a gridworld game board that
  define the logic of the game.
  Board state is tracked as a triple (pieces, scores, rounds_left)
  pieces: batch_size x n_rows x n_cols np.array
  scores: batch_size np.array
  rounds_left: batch_size np.array

  Pieces are interpreted as:
  1=critter, -1=food, 0=empty

  First dim is batch, second dim row , third is col, so pieces[0][1][7]
  is the square in row 2, in column 8 of the first board in the batch of boards

  Note:
    In 2d np.array first dim is row (vertical), second dim is col (horizontal),
    i.e. top left corner is (0,0), so take care when visualizing/plotting
    as np.array visualization inline with typical tensor notation but at odds
    with conventional plotting where (0,0) is bottom left, first dim, x, is
    horizontal, second dim, y, is vertical
  """
  def __init__(self, batch_size=1,
               n_rows=7, n_cols=7,
               num_food=10, lifetime=30):
    """Set the parameters of the game."""
    self.n_rows = n_rows
    self.n_cols = n_cols
    self.batch_size = batch_size
    self.num_food = num_food
    self.lifetime = lifetime


  def get_init_board_state(self):
    """Set up starting board using game parameters"""
    #set rounds_left and score
    self.rounds_left = np.ones(self.batch_size) * self.lifetime
    self.scores = np.zeros(self.batch_size)
    # create an empty board array.
    self.pieces = np.zeros((self.batch_size, self.n_rows, self.n_cols))
    # Place critter and initial food items on the board randomly
    for ii in np.arange(self.batch_size):
      # num_food+1 because we want critter and food locations
      int_loc, rc_idx, rc_plot = init_loc(self.n_rows,
                                          self.n_cols,
                                          self.num_food+1)
      # critter random start location
      self.pieces[(ii, rc_idx[0][0], rc_idx[1][0])] = 1
      # food random start locations
      self.pieces[(ii, rc_idx[0][1:], rc_idx[1][1:])] = -1
    return(self.pieces.copy(), self.scores.copy(), self.rounds_left.copy())


  def set_state(self, board):
    """ board is a triple of np arrays
    pieces,       - batch_size x n_rows x n_cols
    scores,       - batch_size
    rounds_left   - batch_size
    """
    self.pieces = board[0].copy()
    self.scores = board[1].copy()
    self.rounds_left = board[2].copy()


  def get_state(self):
    """ returns a board state, which is a triple of np arrays
    pieces,       - batch_size x n_rows x n_cols
    scores,       - batch_size
    rounds_left   - batch_size
    """
    return(self.pieces.copy(), self.scores.copy(), self.rounds_left.copy())


  def __getitem__(self, index):
    return self.pieces[index]


  def execute_moves(self, moves):
    """
    Updates the state of the board given the moves made.

    Args:
      moves: a 3-tuple of 1-d arrays each of length batch_size,
        the first array gives the specific board within the batch,
        the second array in the tuple gives the new x coord for each critter
        on each board and the third gives the new y coord.

    Note:
      Assumes that there is exactly one valid move for each board in the
      batch of boards. i.e. it does't check for bounce/reflection on edges,
      or for multiple move made on the same board. It only checks for eating
      food and adds new food when appropriate. Invalid moves could lead to
      illegal teleporting behavior, critter dublication, or index out of range
      errors.
    This assumes the move is valid, i.e. doesn't check for
    bounce/reflection on edges, it only checks eating and adds new food,
    so invalid moves could lead to illegal teleporting behaviour or index out
    of range errors
    """
    assert len(moves[0]) == self.pieces.shape[0]
    #critters leave their spots
    self.pieces[self.pieces==1] = 0
    #which critters have food in their new spots
    eats_food = self.pieces[moves] == -1
    # some critters eat and their scores go up
    self.scores = self.scores + eats_food

    num_empty_after_eat = self.n_rows*self.n_cols - self.num_food
    # -1 for the critter +1 for food eaten
    # which boards in the batch had eating happen
    g_eating = np.where(eats_food)[0]
    new_food_sample = rng.choice(num_empty_after_eat, size=np.sum(eats_food))
    # add random food to replace what is eaten
    if np.any(eats_food):
      possible_new_locs = np.where(np.logical_and(
          self.pieces == 0, #the spot is empty
          eats_food.reshape(self.batch_size, 1, 1))) #food eaten on that board
      food_sample_ = rng.choice(num_empty_after_eat, size=np.sum(eats_food))
      food_sample = food_sample_ + np.arange(len(g_eating))*num_empty_after_eat
      assert np.all(self.pieces[(possible_new_locs[0][food_sample],
                                 possible_new_locs[1][food_sample],
                                 possible_new_locs[2][food_sample])] == 0)
      #put new food on the board
      self.pieces[(possible_new_locs[0][food_sample],
                   possible_new_locs[1][food_sample],
                   possible_new_locs[2][food_sample])] = -1
    # put critters in new positions
    self.pieces[moves] = 1.0
    self.rounds_left = self.rounds_left - 1
    assert np.all(self.pieces.sum(axis=(1,2)) == ((self.num_food * -1) + 1))


  def get_legal_moves(self):
    """Identifies all legal moves for the critter, taking into acount
    bouncing/reflection at edges,

    Returns:
      A numpy int array of size batch x 3(g,x,y) x 4(possible moves)

    Note:
      moves[0,1,3] is the x coordinate of the move corresponding to the
      fourth offstet on the first board.
      moves[1,:,1] will give the g,x,y triple corresponding to the
      move on the second board and the second offset, actions are integers
    """

    #apply all possible offsets to each game
    moves = np.stack([
      np.array(np.where(self.pieces == 1)) +
        np.array([np.array([0,  1, 0])]*self.batch_size).T,
      np.array(np.where(self.pieces == 1)) +
        np.array([np.array([0, -1, 0])]*self.batch_size).T,
      np.array(np.where(self.pieces == 1)) +
        np.array([np.array([0, 0,  1])]*self.batch_size).T,
      np.array(np.where(self.pieces == 1)) +
        np.array([np.array([0, 0, -1])]*self.batch_size).T]).swapaxes(0,2)

    #check bounces at boundaries
    moves[:,1,:] = np.where(moves[:,1,:] >=
                            self.n_rows, self.n_rows-2, moves[:,1,:])
    moves[:,2,:] = np.where(moves[:,2,:] >=
                            self.n_cols, self.n_cols-2, moves[:,2,:])
    moves[:,1,:] = np.where(moves[:,1,:] < 0, 1, moves[:,1,:])
    moves[:,2,:] = np.where(moves[:,2,:] < 0, 1, moves[:,2,:])
    return moves




class GridworldGame():
  """
  A collection methods and parameters of a gridworld game that allow
  for interaction with and display of GridwordlBoard objects.
  """
  square_content = {
      -1: "X", #Food
      +0: "-", #Nothing
      +1: "O"  #Critter
      }


  def get_square_piece(self, piece):
    return GridworldGame.square_content[piece]


  def __init__(self, batch_size, n_rows, n_cols, num_food, lifetime):
    self.batch_size = batch_size
    self.n_rows = n_rows
    self.n_cols = n_cols
    self.num_food = num_food
    self.lifetime = lifetime


  def get_init_board(self):
    """
    Generates a starting board given the parameters of the game.
    Returns a tuple giving current state of the game
    """
    # current score, and rounds left in the episode
    b = GridworldBoard(self.batch_size, self.n_rows, self.n_cols,
                       self.num_food, self.lifetime)
    return b.get_init_board_state()


  def get_board_size(self):
    """Shape of a sinlge board, doesn't give batch size"""
    return (self.n_rows, self.n_cols)


  def get_action_size(self):
    """
    Returns the number of all possible actions, even though only  2-4 of
    these will ever be valid on a given turn.
    Actions correspond to integer indexes of board locations,
    moves to g,x,y coordinate indexes of board locations
    """
    return self.n_rows * self.n_cols


  def get_batch_size(self):
    """
    Returns the number of actions, only 2-4 of these will ever be valid.
    Actions correspond to integer indexes of board locations,
    moves to x,y indexes of board locations
    """
    return self.batch_size


  def string_rep(self, board, g):
    """ A bytestring representation board g's state in the batch of boards"""
    return board[0][g].tobytes() + board[1][g].tobytes() + board[2][g].tobytes()


  def string_rep_readable(self, board, g):
    """ A human readable representation of g-th board's state in the batch"""
    board_s = "".join([self.square_content[square] for row in board[0][g]
                       for square in row])
    board_s = board_s + '_' + str(board[1][g])
    board_s = board_s + '_' + str(board[2][g])
    return board_s


  def get_scores(self, board):
    return board[1].copy()


  def get_rounds_left(self, board):
    return board[2].copy()


  def display(self, board, g):
    """Dispalys the g-th games in the batch of boards"""
    print("   ", end="")
    for c_ in range(self.n_cols):
      print(c_, end=" ")
    print("")
    print("-----------------------")
    for c_ in range(self.n_cols):
      print(c_, "|", end="")    # Print the row
      for r_ in range(self.n_rows):
        piece = board[0][g,c_,r_]    # Get the piece to print
        print(GridworldGame.square_content[piece], end=" ")
      print("|")
    print("-----------------------")
    print("Rounds Left: " + str(board[2][g]))
    print("Score: " + str(board[1][g]))


  def get_critter_rc(self, board, g):
    return np.squeeze(np.array(np.where(board[0][g]==1)))


  def plot_board(self, board, g,
                 fig=None, ax=None, critter=None, food=None):
    """Uses plotting functions to make picture of the current board state"""
    batch_size, n_rows, n_cols = board[0].shape
    plt.ioff()
    if fig is None and ax is None:
      fig, ax = make_grid(n_rows, n_cols)
    rc_critter = self.get_critter_rc(board, g)
    if critter is None:
      critter = plot_critter(fig, ax, rc_critter)
    else:
      critter = plot_critter(fig, ax, rc_critter, critter)
    rc_food_index = np.array(np.where(board[0][g] == -1))
    rc_food_plotting = np.array(rc_food_index).T
    if food is None:
      food = plot_food(fig, ax, rc_food_plotting)
    else:
      food = plot_food(fig, ax, rc_food_plotting, food)
    fig.legend(loc = "outside right upper")
    #fig.canvas.header_visible = False
    #fig.canvas.toolbar_visible = False
    #fig.canvas.resizable = False
    #fig.canvas.footer_visible = False
    fig.canvas.draw()
    return fig, ax, critter, food


  def get_valid_actions(self, board):
    """
    A Helper function to translate the g,x,y, tuples provided the
    GridworldBoard.get_legal_moves method into valid actions, represented
    as binary vectors of len num_actions.

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size

    Returns:
      valids: np.ndarray(binary) batch_size x num_actions, 1's represent
              valid moves
    """
    batch_size, n_rows, n_cols = board[0].shape
    b = GridworldBoard(batch_size, n_rows, n_cols,
                       self.num_food, self.lifetime)
    b.set_state(board)
    legal_moves =  b.get_legal_moves()
    valids = np.zeros((batch_size, n_rows * n_cols))
    for ii, g in enumerate(legal_moves[:,1:,:]):
      for x,y in zip(g[0],g[1]):
        valids[ii, x*self.n_cols+y] = 1
    return valids


  def display_moves(self, board, g):
    """Dispaly possible moves for the g-th games in the batch of boards"""
    batch_size, n_rows, n_cols = board[0].shape
    A=np.reshape(self.get_valid_actions(board)[g], (n_rows, n_cols))
    print("  ")
    print("possible moves")
    print("   ", end="")
    for y in range(self.n_cols):
      print(y, end=" ")
    print("")
    print("-----------------------")
    for y in range(self.n_cols):
      print(y, "|", end="")    # Print the row
      for x in range(self.n_rows):
        piece = A[y][x]    # Get the piece to print
        print(GridworldGame.square_content[piece], end=" ")
      print("|")
    print("-----------------------")


  def get_next_state(self, board, actions, a_indx=None):
    """
    Helper function using GridworldBoard.execute_moves to update board state
    given actions on a batch of boards

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
      actions: list of integer indexes of critter's new board positions
      a_indx: list of integer indexes indicating which actions are being taken
        on which boards in the batch

    Returns:
      a board tiple signifying next state

    Note:
      if len(actions) > batch_size of board the returned board state will have
      an expanded batch size, allowing multiple paths in the game tree to be
      explored in parellel

    """
    batch_size, n_rows, n_cols = board[0].shape
    if board[2][0] <= 0:
      # assumes all boards in the batch have the same rounds left
      # no rounds left return the board unchanged
      return board
    else:
      moves = (np.arange(len(actions)),
               np.floor_divide(actions, n_cols),
               np.remainder(actions, n_cols))
      b = GridworldBoard(len(actions), n_rows, n_cols,
                         self.num_food, self.lifetime)
      if a_indx is None:
        # just one move on each board in the batch
        assert batch_size == len(actions)
        b.set_state(board)
      else:
        # potentially multiple moves on each board, expand the batch
        assert len(actions) == len(a_indx)
        newPieces = np.array([board[0][ai].copy() for ai in a_indx])
        newScores = np.array([board[1][ai].copy() for ai in a_indx])
        newrounds_left = np.array([board[2][ai].copy() for ai in a_indx])
        b.set_state((newPieces, newScores, newrounds_left))
      b.execute_moves(moves)
      return b.get_state()


  def critter_oriented_get_next_state(self, board, offsets):
    """
    Translates directions in reference to the critter's location into
    moves on the board in absolute terms, while checking for
    bouncing/reflecting then get's the next state.

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
      offsets: batch length list of strings one 'up', 'down', 'left', 'right'

    Returns:
      a board triple signifying next state

    Note:
      Unlike get_next_state, this method does not allow for expansion
      of the game tree, i.e. len(offsets)==batch_size required
    """
    assert len(offsets) == board[0].shape[0]
    batch_size, n_rows, n_cols = board[0].shape
    b = GridworldBoard(batch_size, n_rows, n_cols,
                       self.num_food, self.lifetime)
    b.set_state(board)
    offset_dict = {'up': (0, -1, 0),
                   'down': (0, 1, 0),
                   'left': (0, 0, -1),
                   'right': (0, 0, 1)}
    critter_locs = np.where(board[0] == 1)
    offsets_array = np.hstack([np.array(offset_dict[ost_]).reshape((3,1))
                           for ost_ in offsets])
    new_locs = np.array(critter_locs) + offsets_array
    #check bounces at boundaries
    new_locs[1,:] = np.where(new_locs[1] >=
                               n_rows, n_rows-2, new_locs[1])
    new_locs[2,:] = np.where(new_locs[2,:] >=
                               n_cols, n_cols-2, new_locs[2,:])
    new_locs[1,:] = np.where(new_locs[1,:] < 0, 1, new_locs[1,:])
    new_locs[2,:] = np.where(new_locs[2,:] < 0, 1, new_locs[2,:])
    moves = tuple(new_locs)
    b.execute_moves(moves)
    return(b.get_state())


  def get_game_ended(self, board):
    """
    Helper function to signify if game has ended

    Args:
      board: a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size

    Returns:
      batch size np.array of -1 if not ended, and scores for
      each game in the batch if it is ended
    """
    rounds_left = board[2]
    scores = board[1]
    if np.any(rounds_left >= 1):
      return np.ones(self.batch_size) * -1.0
    else:
      return scores




class Timer:
  def __init__(self, timeout, callback):
    self._timeout = timeout
    self._callback = callback
  async def _job(self):
    await asyncio.sleep(self._timeout)
    self._callback()
  def start(self):
    self._task = asyncio.ensure_future(self._job())
  def cancel(self):
    self._task.cancel()




class InteractiveGridworld():
  """
  A widget based tool for interating with a gridworld game
  """

  def __init__(self, gridworld_game, init_board=None):
    """
    Initializes a widget based object for interacting with a gridworld game

    Args:
      gridworld_game: an instance of GridworldGame object
      init_board: (optional) a triple of np arrays representing board state
        pieces,       - batch_size x n_rows x n_cols
        scores,       - batch_size
        rounds_left   - batch_size
        if left out will initialize with a random board state
    """
    # Set GridworldGame object and initialize the board state
    self.gwg = gridworld_game
    self.final_scores = []
    if init_board is None:
      self.board_state = self.gwg.get_init_board()
    else:
      self.board_state = init_board

    # Initialize widgets and buttons
    self.output = widgets.Output()
    self.scoreboard = widgets.Output()
    self.up_button = widgets.Button(description="Up")
    self.down_button = widgets.Button(description="Down")
    self.left_button = widgets.Button(description="Left")
    self.right_button = widgets.Button(description="Right")
    self.random_movement = widgets.Checkbox( value=False,
                                            description='Move Randomly',
                                             disabled=False,
                                             indent=False)
    # Connect the buttons to functions that do something
    self.up_button.on_click(self.on_up_button_clicked)
    self.down_button.on_click(self.on_down_button_clicked)
    self.left_button.on_click(self.on_left_button_clicked)
    self.right_button.on_click(self.on_right_button_clicked)

    self.buttons = widgets.HBox([self.left_button,
                               widgets.VBox([self.up_button, self.down_button]),
                               self.right_button])
    plt.ioff()
    (self.b_fig, self.b_ax,
     self.b_critter, self.b_food) = self.gwg.plot_board(self.board_state, 0)
    self.board_and_output = widgets.VBox([self.b_fig.canvas, self.output])
    self.board_and_buttons = widgets.VBox([self.board_and_output,
                                            self.buttons])
    self.board_output_and_score = widgets.HBox([self.board_and_output,
                                                self.scoreboard])
    self.boards_buttons_and_score = widgets.HBox([self.board_and_buttons,
                                                  self.scoreboard])

    # Sometimes use timer
    self.click_timer = Timer(5.0, self.random_click)


  def button_output_update(self, which_button):
    old_board = tuple([self.board_state[ii].copy() for ii in range(3)])
    old_score = old_board[1][0]
    self.board_state = self.gwg.critter_oriented_get_next_state(
        self.board_state, [which_button])
    new_score = self.board_state[1][0]
    rounds_left = self.board_state[2][0]
    num_moves = self.gwg.lifetime - rounds_left
    if new_score > old_score:
      eating_string = "The critter ate the food there!"
    else:
      eating_string = "There's no food there."
    row, col = self.gwg.get_critter_rc(self.board_state, 0)
    (self.b_fig, self.b_ax,
     self.b_critter, self.b_food) = self.gwg.plot_board(
        self.board_state, 0, self.b_fig, self.b_ax, self.b_critter, self.b_food)
    with self.output:
      #display(self.b_fig.canvas)
      clear_output()
      print("Critter moved " + which_button +
            " and is now at ({}, {}).".format(row,col))
      print(eating_string)
      print("Rounds Left: {} \tFood Eaten: {} \tFood Per Move: {:.2f}".format(
          rounds_left, new_score, new_score / num_moves))
      #if self.random_movement.value == True:
      #  self.click_timer.start()
    if rounds_left == 0:
      self.final_scores.append(new_score)
      with self.output:
        clear_output
        print('Game Over. Final Score {}'.format(new_score))
        print('Resetting the board for another game')
        self.board_state = self.gwg.get_init_board()
      (self.b_fig, self.b_ax,
       self.b_critter, self.b_food) = self.gwg.plot_board(
        self.board_state, 0, self.b_fig, self.b_ax, self.b_critter, self.b_food)
    with self.scoreboard:
        clear_output()
        print('Games Played: ' + str(len(self.final_scores)))
        if len(self.final_scores) > 0:
          print('High Score: ' + str(np.max(np.array(self.final_scores))))
          print('Last Score: ' + str(self.final_scores[-1]))
          print('Average Score: {:.2f}'.format(np.mean(self.final_scores)))
        else:
          print('High Score: --')
          print('Last Score: --')
          print('Average Score: --')


  def on_up_button_clicked(self, *args):
    self.button_output_update('up')

  def on_down_button_clicked(self, *args):
    self.button_output_update('down')

  def on_left_button_clicked(self, *args):
    self.button_output_update('left')

  def on_right_button_clicked(self, *args):
    self.button_output_update('right')


  def random_click(self):
    move = np.random.randint(0,4)
    if move == 0:
      self.up_button.click()
    elif move == 1:
      self.down_button.click()
    elif move == 2:
      self.left_button.click()
    elif move == 3:
      self.right_button.click()
    else:
      print('should not happen')

In [ ]:
# @title Other Helper Functions
# @markdown You don't need to worry about how this code works – but you do need to **run the cell**

class dotdict(dict):
  def __getattr__(self, name):
    return self[name]

def content_review(notebook_section: str):
  return DatatopsContentReviewContainer(
    "",  # No text prompt
    notebook_section,
    {
      "url": "https://pmyvdlilci.execute-api.us-east-1.amazonaws.com/klab",
      "name": "neuro_book",
      "user_key": "xuk960xj",
    },
  ).render()

# Micro 1.1.1.1: Initializing Gridworld

Before we introduce an organism with **behaviour** we're going to build an **environment** for them to behave in. To start, this world will consist of a 7 x 7 grid of cells. Let's make a picture of that and see what it looks like.

In [ ]:
############################################################################
## TO DO for students: replace ... with the correct arguments(inputs) is the
## make_grid function below to a grid the right size and shape. You can use the
## tool tip by hovering over the word make_grid to find out how to use it. You
## can also use the tool tip to view the source code. How does it work?
raise NotImplementedError("Student exercise: make grid using the make_grid function")
############################################################################

fig, ax = make_grid(...)

[*Click for solution*](https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/tree/main//sequences/P1C1_BehaviourAsPolicy/solutions/P1C1_Sequence1_Solution_c2694372.py)

*Example output:*

<img alt='Solution hint' align='left' width=700.0 height=600.0 src=https://raw.githubusercontent.com/dcownden/PerennialProblemsOfLifeWithABrain/main/sequences/P1C1_BehaviourAsPolicy/static/P1C1_Sequence1_Solution_c2694372_0.png>



*Bonus*: Tweak the make_grid function in the Plotting Functions cell above to make the grid lines green.

Wow, what a boring environment. Let's add an organism and something for that organism to interact with. We'll start with 10 food items scattered randomly throughout the grid, never more than one food item per cell. To plot these food items we need their locations. We will set these by randomly sampling grid coordinates [without replacement](## "never picking the same (row,col) coordinate pair twice"). We'll place the organism in the same way and not on a food item to start.

In [ ]:
################################################################################
# TODO for students: replace ... in init_loc(...) to initialize the right
# number of food item locations and critter locations in coordinates that make
# sense for our grid environment. Then replace the ... in rc_plotting[...] to
# index the plotting coordinates for the food locations.
# then comment out or remove the next line.
raise NotImplementedError("Exercise: initialize food and critter locations")
################################################################################
def init_loc(n_rows, n_cols, num):
  """
  Samples random 2d grid locations without replacement

  Args:
    n_rows: int
    n_cols: int
    num:    int, wnumber of samples to generate, should
            throw an error ifnum <= n_rows x n_cols

  Returns:
    int_loc: ndarray(int) of flat indices for a grid
    rc_index: (ndarray(int), ndarray(int)) a pair of arrays the first
      giving the row indices, the second giving the col indices, useful
      for indexing an n_rows by n_cols numpy array
    rc_plotting: ndarray(int) num x 2, same rc coordinates but structured
      in the way that matplotlib likes
  """
  int_loc = np.random.choice(n_rows * n_cols, num, replace=False)
  rc_index = np.unravel_index(int_loc, (n_rows, n_cols))
  rc_plotting = np.array(rc_index).T
  return int_loc, rc_index, rc_plotting


with plt.xkcd():
  fig, ax = make_grid(7, 7)
  int_locs, rc_index, rc_plotting = init_loc(...)

  rc_critter = (rc_plotting[0])
  plot_critter(fig, ax, rc_critter)

  rc_food = rc_plotting[...]
  plot_food(fig, ax, rc_food)

  fig.legend(loc='outside right upper')

[*Click for solution*](https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/tree/main//sequences/P1C1_BehaviourAsPolicy/solutions/P1C1_Sequence1_Solution_c438b3af.py)

*Example output:*

<img alt='Solution hint' align='left' width=700.0 height=600.0 src=https://raw.githubusercontent.com/dcownden/PerennialProblemsOfLifeWithABrain/main/sequences/P1C1_BehaviourAsPolicy/static/P1C1_Sequence1_Solution_c438b3af_0.png>



In [ ]:
# @markdown Vibe Check
content_review("Sequence 1.1.1 Micro 1")

---
# Micro 1.1.1.2: Random Eating

Now that we have an environment scattered with food and an organism, let's introduce some behaviour. The organism drifts around the environment randomly and eats the food it happens to stumble upon. (Can you think of any organisms that employ this strategy?). When food is eaten, the organism gets a **reward**, in this case a *Food Eaten* point, and a new food item appears randomly somewhere else in the environment (that doesn't already have food). Run the code cell below to see what this looks like.

In [ ]:
# @title Random Movement
# @markdown You don't need to worry about how this code works, yet – just **run this cell** and the one below for now and watch what happens

gwg = GridworldGame(1, 7, 7, 10, 30)
random_igwg = InteractiveGridworld(gwg)
display(random_igwg.b_fig.canvas)
clear_output()
display(random_igwg.board_output_and_score)

In [ ]:
# @markdown Run this cell again to watch the random organism play again
for ii in range(30):
  time.sleep(0.1)
  random_igwg.random_click()

*Question:* When the organism is just drifting around randomly how good is it at eating lots of food, what is its efficiency in terms of food per movement? Now run the cell above a few more times. Does the organism always eat the same amount of food or does it change between simulation runs?

[*Click for solution*](https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/tree/main//sequences/P1C1_BehaviourAsPolicy/solutions/P1C1_Sequence1_Solution_82f98983.py)



Before we move on it's important to test that our simulation is running as we expect. Randomness can make testing hard, but can overcome in part by setting up the environment in such a way that the outcome becomes deterministic. In the two cells bellow change how the the Gridworld is initialized. By alterning the size, shape and number of food items available create a scenario where the organism will always achieve perfect efficnency and a scenario where the organism will fail completely.

In [ ]:
###############################################################################
# TODO for students: replace the ...'s in GridworldGame(...) to initialize a
# grid world where the organism is always 100% efficient.
raise NotImplementedError("Exercise: make random movement 100% efficient")
################################################################################

gwg = GridworldGame(1, ..., ..., ..., 30)
random_igwg_100 = InteractiveGridworld(gwg)
display(random_igwg_100.b_fig.canvas)
clear_output()
display(random_igwg_100.board_output_and_score)

[*Click for solution*](https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/tree/main//sequences/P1C1_BehaviourAsPolicy/solutions/P1C1_Sequence1_Solution_4ef49ade.py)



In [ ]:
for ii in range(30):
  time.sleep(0.1)
  random_igwg_100.random_click()

In [ ]:
###############################################################################
# TODO for students: replace the ...'s in GridworldGame(...) to initialize a
# grid world where the organism is always 0% efficient.
raise NotImplementedError("Exercise: make random movement 0% efficient")
################################################################################

gwg = GridworldGame(1, ..., ..., ..., 30)
random_igwg_0 = InteractiveGridworld(gwg)
display(random_igwg_0.b_fig.canvas)
clear_output()
display(random_igwg_0.board_output_and_score)

[*Click for solution*](https://github.com/dcownden/PerennialProblemsOfLifeWithABrain/tree/main//sequences/P1C1_BehaviourAsPolicy/solutions/P1C1_Sequence1_Solution_9d770cf4.py)



In [ ]:
for ii in range(30):
  random_igwg_0.random_click()
  time.sleep(0.1)

In [ ]:
# @markdown Vibe Check
content_review("Sequence 1.1.1 Micro 2")

---
# Micro 1.1.1.3: Better Than Random Eating
Now it's your turn to control the organism. Run the next cell and see how much more efficient than random drifting your control of the organism is in terms of food per movement.

In [ ]:
# @title Controlled Movement
# @markdown You don't need to worry about how this code works – just **run the cell** and then use the buttons to guide the organism

# user in control
gwg = GridworldGame(1, 7, 7, 10, 30)
user_control_igwg = InteractiveGridworld(gwg)
display(user_control_igwg.b_fig.canvas)
clear_output()
display(user_control_igwg.boards_buttons_and_score)

Hopefully your performance was more successful than random flailing. Even in this relatively simple and contrived foraging scenario intelligence can help a lot. What kinds of strategies and heuristics did you use to guide your choice of direction? Essentially, the fundamental purpose of a nervous system and a brain is to solve problems of this kind—deciding which actions to take based on environmental inputs to maximize rewards.

In [ ]:
# @markdown Vibe Check
content_review("Sequence 1.1.1 Micro 3")

---
# Micro 1.1.1.4: Optimal Eating
Finally we'd like to introduce a time traveling super organism, GW7x7-10-30, from the last chapter of this book. GW7x7-10-30 has absolutely mastered 7x7 Gridworld, with 10 food items and a 30 round duration. Run the next cell and see how efficient a highly optimized and specialized Gridworld organism can be. Are you as good as gw7x7-10-30? If not, you gotta read this book 😉 (If you can't beat the AIs, at least learn how to program them.)

(Last chapter not made yet so working on a minimal sketch/skeleton version to produce this agent)

In [ ]:
# @title Optimal Movement
# @markdown You don't need to worry about how this code works – just **run the cell** to watch the super organism behave. See if you can outperform them.

# master organism in control

In [ ]:
# @markdown Vibe Check
content_review("Sequence 1.1.1 Micro 4")

---
# Graveyard


Useful scavenging from github and SO

In [ ]:
from matplotlib import animation, rc, patches
from IPython.display import HTML

In [ ]:
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xticks(np.arange(0, 11, 1))
ax.set_yticks(np.arange(0, 11, 1))

# Labels for major ticks
ax.set_xticklabels(np.arange(0, 11, 1))
ax.set_yticklabels(np.arange(0, 11, 1))

# Minor ticks
ax.set_xticks(np.arange(1.5, 10.5, 1), minor=True)
ax.set_yticks(np.arange(1.5, 10.5, 1), minor=True)

# Gridlines based on minor ticks
ax.grid(which='minor', color='grey', linestyle='-', linewidth=2, alpha=0.5)

# Remove minor ticks
ax.tick_params(which='minor', bottom=False, left=False)

ax.set_xlim(( 0.5, 10.5))
ax.set_ylim(( 0.5, 10.5))

#ax.axis("equal")
scatter = ax.scatter([], [])
#critterCircle = plt.Circle((5,5), 0.4)
#dot = ax.add_patch(critterCircle)
#scatter.set_offsets([[5, 5]])

scatter.set_offsets([[3, 11-3]])
#fig.canvas.draw()
#fig.canvas.flush_events()

In [ ]:
def init():
    scatter.set_offsets([[5, 5]])
    return scatter,

In [ ]:
def animate_dot(i):
    print(i)
    x = i
    y = 11-i
    scatter.set_offsets([[x, y]])
    return scatter,

In [ ]:
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate_dot, init_func=init,
                               frames=(np.arange(10)+1), interval=500, blit=True)

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
rc('animation', html='html5')

In [ ]:
anim

These movies are a bit slicker than drawing using the matplotlib figure canvas updates, is this something we can/should incorporate into the non-interactive versions where you just watch something play.

Thoughts to tweak the game, have random movement be something that always happens and directed movement as an optional intervention, maybe on a timer?

